<a href="https://colab.research.google.com/github/Akashgowtham1/Hate-Speech-Detetection/blob/main/HateSpeechDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Hate Speech Detection


In [1]:
#Importing Pandas and re library
import pandas as pd
import re

In [2]:
#Loading the Training dataset
train = pd.read_csv('train.csv')

In [3]:
#Loading the Test dataset
test = pd.read_csv('test.csv')

In [4]:
#Defining a function to return a clean text
def  cleaning_text(df, text_field):
    df[text_field] = df[text_field].str.lower()
    df[text_field] = df[text_field].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  
    return df


In [5]:
#Cleaning the test and training dataset
test_clean  = cleaning_text(test, "tweet")
train_clean = cleaning_text(train,"tweet")

In [6]:
#Importing resample from sklearn library
from sklearn.utils import resample
train_major = train_clean[train_clean.label==0]
train_minor = train_clean[train_clean.label==1]
#Here we are upsampling the data
train_minor_upsampled = resample(train_minor,replace=True,n_samples=len(train_major),random_state=123)
train_upsampled = pd.concat([train_minor_upsampled, train_major])
#The upsampled data
train_upsampled['label'].value_counts()

1    29720
0    29720
Name: label, dtype: int64

In [7]:
#Importing several modules from sklearn library
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier


In [8]:
#Here we are building the pipeline of Scikit learn SGDClassifier
pipeline_SGD = Pipeline([('vect', CountVectorizer()),('tfidf',  TfidfTransformer()),('nb', SGDClassifier()),])

In [9]:

#Importing the library for splitting test and training data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_upsampled['tweet'],train_upsampled['label'],random_state = 0)

In [10]:
#Training our model
model = pipeline_SGD.fit(X_train, y_train)
y_predict = model.predict(X_test)

In [12]:
#Calculating f1_score using the sklearn library
from sklearn.metrics import f1_score
f1_score(y_test, y_predict)

0.9696